In [1]:
import pandas as pd

# set your path
path = "ml-100k/"

# load ratings
ratings = pd.read_csv(path + "u.data", sep="\t", header=None,
                      names=["userId", "movieId", "rating", "timestamp"])

# load movies
movies = pd.read_csv(path + "u.item", sep="|", header=None,
                     names=["movieId", "title", "release_date", "video_release_date",
                            "IMDb_URL"] + [f"genre_{i}" for i in range(19)],
                     encoding="latin-1")

# load users
users = pd.read_csv(path + "u.user", sep="|", header=None,
                    names=["userId", "age", "gender", "occupation", "zip_code"])


In [2]:
ratings["timestamp"] = pd.to_datetime(ratings["timestamp"], unit="s")


In [4]:
data = ratings.merge(movies[["movieId","title"]], on="movieId", how="left")
data.head()

,userId,movieId,rating,timestamp,title
0,196,242,3,1997-12-04 15:55:49,Kolya (1996)
1,186,302,3,1998-04-04 19:22:22,L.A. Confidential (1997)
2,22,377,1,1997-11-07 07:18:36,Heavyweights (1994)
3,244,51,2,1997-11-27 05:02:03,Legends of the Fall (1994)
4,166,346,1,1998-02-02 05:33:16,Jackie Brown (1997)


In [5]:
min_ratings_per_user = 5
min_ratings_per_movie = 5

# filter users
user_counts = data["userId"].value_counts()
active_users = user_counts[user_counts >= min_ratings_per_user].index
data = data[data["userId"].isin(active_users)]

# filter movies
movie_counts = data["movieId"].value_counts()
popular_movies = movie_counts[movie_counts >= min_ratings_per_movie].index
data = data[data["movieId"].isin(popular_movies)]


In [7]:
!pip install scikit-surprise


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build scikit-surprise


  error: subprocess-exited-with-error
  
  × Building wheel for scikit-surprise (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [155 lines of output]
      C:\Users\Sri Raam\AppData\Local\Temp\pip-build-env-qp_6dh8y\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              *************************************

In [10]:
import surprise
print("Surprise installed!")

from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1,5))
dataset = Dataset.load_from_df(data[["userId","movieId","rating"]], reader)
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)

algo = SVD(n_factors=50, random_state=42)
algo.fit(trainset)
predictions = algo.test(testset)


ModuleNotFoundError: No module named 'surprise'